Importations

In [1]:
!pip install sentence-transformers qdrant-client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully 

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models

**Chargement des données**

In [ ]:
df = pd.read_csv('votre_fichier_bible.csv')  # Remplacez par le chemin de votre fichier

**Génération des embeddings**

In [ ]:
encoder = SentenceTransformer("BAAI/bge-m3")  # Remplacez par le modèle de votre choix
df['text_chunks'] = df['Text'].apply(lambda x: x.split('. '))  # Divisez le texte en morceaux

**Stochage des embeddings**

In [ ]:
client = QdrantClient(url="http://localhost:6333")  # Changez l'URL si nécessaire

**Création d'une collection dans Qdrant**

In [ ]:
client.create_collection(
    collection_name="bible",
    vectors_config=models.VectorParams(
        distance=models.Distance.COSINE,
        size=encoder.get_sentence_embedding_dimension()  # Taille des embeddings
    ),
)

**Créer les points et les téléverser**

In [ ]:

# Créer les points et les téléverser
points = []
point_id = 1

for _, row in df.iterrows():
    for chunk in row['text_chunks']:
        vector = encoder.encode(chunk)  # Encoder chaque morceau
        payload = {
            'Book Name': row['Book Name'],
            'Chapter': row['Chapter'],
            'Verse': row['Verse'],
            'Text': chunk  # Utilisez le morceau de texte
        }
        points.append(
            models.PointStruct(
                id=point_id,  # ID unique pour chaque point
                vector=vector.tolist(),  # Convertir en liste
                payload=payload
            )
        )
        point_id += 1  # Incrémenter l'ID

# Téléverser les points dans la collection
client.upload_points(
    collection_name="bible",
    points=points,
)

print("Les embeddings ont été générés et sauvegardés dans Qdrant.")

**Recherche des voisins les plus proches**

In [ ]:
def search_bible(question, k=5):
    question_embedding = encoder.encode(question)  # Encoder la question
    D, I = client.search(
        collection_name="bible",
        query_vector=question_embedding.tolist(),
        limit=k
    )

    results = []
    for distance, index in zip(D[0], I[0]):
        result = client.get_point(collection_name="bible", point_id=index)
        results.append({
            'distance': distance,
            'Book Name': result.payload['Book Name'],
            'Chapter': result.payload['Chapter'],
            'Verse': result.payload['Verse'],
            'Text': result.payload['Text']
        })

    return results

In [ ]:
question = "Quel est le sens de la vie selon la Bible ?"
results = search_bible(question)
for result in results:
    print(f"Book: {result['Book Name']}, Chapter: {result['Chapter']}, Verse: {result['Verse']}, Text: {result['Text']}, Distance: {result['distance']:.4f}")